In [23]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
feature_sets = []
feature_set_labels = []

# import three base feature sets
feature_sets.append(pd.read_csv('C:/Users/cqyzxy/Downloads/Stoich45_PCA_dataset.csv', sep=','))
feature_set_labels.append('Stoich45 PCs')
# ... rename principal component columns to be more descriptive
feature_sets[0] = feature_sets[0].rename(columns = dict([[str(i), 'stoich45 PC '+str(i+1)] for i in range(8)]))

feature_sets.append(pd.read_csv('C:/Users/cqyzxy/Downloads/Stoich45_FeatureSelected_dataset.csv', sep=','))
feature_set_labels.append('Stoich45 intersection')

feature_sets.append(pd.read_csv('C:/Users/cqyzxy/Downloads/SCM_PCA_trainingStoich45_dataset.csv', sep=','))
feature_set_labels.append('SCM PCs')

# merge to form last two
feature_sets.append(feature_sets[0].merge(feature_sets[2]))
feature_set_labels.append('Stoich45 PCs + SCM PCs')

feature_sets.append(feature_sets[1].merge(feature_sets[2]))
feature_set_labels.append('Stoich45 intersection + SCM PCs')

# drop MOF column and rename target in all feature sets
feature_sets = [fs.drop(columns = ['MOF']).rename(columns = {'outputs.hse06.bandgap': 'HSE06 Bandgap'}) for fs in feature_sets]


In [ ]:
target = 'HSE06 Bandgap'
kfold = KFold(n_splits = 4, shuffle = True, random_state = 1234)

def get_mean_cv_mse(model, df_feature_set):
     return -cross_val_score(
         model,
         X = df_feature_set.drop(columns = [target]), y = df_feature_set[target],
         cv = kfold, scoring = 'neg_mean_squared_error',
         n_jobs = 4
     ).mean()

def get_mean_cv_mse_and_frac_error(model, df_feature_set):
    validation_target_means = [
        df_feature_set[target][test_index].mean()
        for (train_index, test_index) in kfold.split(df_feature_set)
    ]
    mses = -cross_val_score(
        model,
        X = df_feature_set.drop(columns = [target]), y = df_feature_set[target],
        cv = kfold, scoring = 'neg_mean_squared_error',
        n_jobs = 4
    )
    return (mses.mean(), (np.sqrt(mses) / validation_target_means).mean())


# Bayesian Ridge Regression

Stoich45_PC

In [ ]:
def powerset(s):
    power_set = [[]]
    for x in s:
        power_set += [s0+[x] for s0 in power_set]
    return power_set[1:]
    
cols_stoich45_PCs_powerset = powerset(feature_sets[0].columns[:-1])
print('number of subsets:', len(cols_stoich45_PCs_powerset))


number of subsets: 255


In [19]:
brr_stoich45_PCs_subsets_cv_mse = np.zeros((len(cols_stoich45_PCs_powerset),))

for (i, col_subset) in enumerate(cols_stoich45_PCs_powerset):
    df_subset = feature_sets[0][col_subset + [target]]
    brr_stoich45_PCs_subsets_cv_mse[i] = get_mean_cv_mse(BayesianRidge(),df_subset)

print('The best performing subset is', cols_stoich45_PCs_powerset[np.argmin(brr_stoich45_PCs_subsets_cv_mse)])

The best performing subset is ['stoich45 PC 1', 'stoich45 PC 2', 'stoich45 PC 3', 'stoich45 PC 4', 'stoich45 PC 5', 'stoich45 PC 6', 'stoich45 PC 7', 'stoich45 PC 8']


In [20]:

bayesridge_cv_mse = np.zeros((len(feature_sets),))
alpha_values = np.zeros((len(feature_sets),))
lambda_values = np.zeros((len(feature_sets),))  # Store lambda values

print(f"{'Feature Set':<35}{'Cross-Validation MSE':<25}{'Estimated Alpha':<20}{'Estimated Lambda'}")

for i, fs in enumerate(feature_sets):
    model = BayesianRidge()
    X = fs.drop(columns=[target])
    y = fs[target]
    
    model.fit(X, y)  # Fit model to extract alpha_ and lambda_
    
    bayesridge_cv_mse[i] = get_mean_cv_mse(model, fs)
    alpha_values[i] = model.alpha_
    lambda_values[i] = model.lambda_  # Extract estimated lambda

    print(f"{feature_set_labels[i]:<35}{round(bayesridge_cv_mse[i], 6):<25}{round(alpha_values[i], 6):<20}{round(lambda_values[i], 6)}")
    


Feature Set                        Cross-Validation MSE     Estimated Alpha     Estimated Lambda
Stoich45 PCs                       0.803307                 1.247673            129.998887
Stoich45 intersection              0.687404                 1.466944            18.760048
SCM PCs                            0.862288                 1.164813            0.00403
Stoich45 PCs + SCM PCs             0.752608                 1.336373            0.011948
Stoich45 intersection + SCM PCs    0.683951                 1.50771             0.047704


$$
\alpha \text{ Precision of noise (inverse of noise variance). A high value suggests low noise in the data.} \\
\lambda \text{ Precision of the weights (inverse of weight variance). A high value suggests stronger regularization, shrinking weights toward zero.}
$$


Stoich45 Int

In [49]:
df_featuresInt = feature_sets[1].iloc[:, :-1]  # Select all columns except the last one

# Create PolynomialFeatures transformer
poly = PolynomialFeatures(degree=2,interaction_only=True,  include_bias=False)
#poly = PolynomialFeatures(degree=2, include_bias=False)

# Transform the features
df_poly = poly.fit_transform(df_featuresInt)
df_poly = pd.DataFrame(df_poly, columns=poly.get_feature_names_out(df_featuresInt.columns))
df_final = pd.concat([feature_sets[1], df_poly], axis=1)


Adding polynomial features


In [ ]:
model = BayesianRidge()
X = df_final.drop(columns=[target])
y = df_final[target]
    
model.fit(X, y)  # Fit model to extract alpha_ and lambda_
    
bayesridge_cv_mse = get_mean_cv_mse(model, df_final)
alpha_value = model.alpha_
lambda_value = model.lambda_  # Extract estimated lambda
print("MSE:", bayesridge_cv_mse)
print("Alpha:", alpha_value)
print("Lambda:", lambda_value)


MSE: 0.6027140409142605
Alpha: 1.7537612608424786
Lambda: 44640737.84842803


Only Interaction Terms

In [50]:
model = BayesianRidge()
X = df_final.drop(columns=[target])
y = df_final[target]
    
model.fit(X, y)  # Fit model to extract alpha_ and lambda_
    
bayesridge_cv_mse = get_mean_cv_mse(model, df_final)
alpha_value = model.alpha_
lambda_value = model.lambda_  # Extract estimated lambda
print("MSE:", bayesridge_cv_mse)
print("Alpha:", alpha_value)
print("Lambda:", lambda_value)


MSE: 0.6133255169888974
Alpha: 1.726234605981679
Lambda: 42807912.76454939


Original data

In [ ]:
model = BayesianRidge()
X = feature_sets[1].drop(columns=[target])
y = feature_sets[1][target]
    
model.fit(X, y)  # Fit model to extract alpha_ and lambda_
    
bayesridge_cv_mse = get_mean_cv_mse(model, feature_sets[1])
alpha_value = model.alpha_
lambda_value = model.lambda_  # Extract estimated lambda
print("MSE:", bayesridge_cv_mse)
print("Alpha:", alpha_value)
print("Lambda:", lambda_value)


MSE: 0.6874039376897668
Alpha: 1.466943966471058
Lambda: 18.760047520564846


SCM PCs

All polynomial features

In [65]:
df_features_scm = feature_sets[2].iloc[:, :-1]  # Select all columns except the last one

# Create PolynomialFeatures transformer
#poly = PolynomialFeatures(degree=2,interaction_only=True,  include_bias=False)
poly = PolynomialFeatures(degree=2, include_bias=False)

# Transform the features
df_poly = poly.fit_transform(df_features_scm)
df_poly = pd.DataFrame(df_poly, columns=poly.get_feature_names_out(df_features_scm.columns))
df_final = pd.concat([feature_sets[2], df_poly], axis=1)

model = BayesianRidge()
X = df_final.drop(columns=[target])
y = df_final[target]
    
model.fit(X, y)  # Fit model to extract alpha_ and lambda_
    
bayesridge_cv_mse = get_mean_cv_mse(model, df_final)
alpha_value = model.alpha_
lambda_value = model.lambda_  # Extract estimated lambda
print("MSE:", bayesridge_cv_mse)
print("Alpha:", alpha_value)
print("Lambda:", lambda_value)


MSE: 0.8099840626942818
Alpha: 1.2613610816066323
Lambda: 1.4938056124882531e-05


In [66]:
coefficients = model.coef_
feature_names = df_final.drop(columns=[target]).columns  
coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})
top_10_features = coef_df.reindex(coef_df.Coefficient.abs().sort_values(ascending=False).index).head(10)
print(top_10_features)


                       Feature  Coefficient
68     sineCM PC 3 sineCM PC 4   736.952420
88    sineCM PC 4 sineCM PC 10   475.562392
75    sineCM PC 3 sineCM PC 11   470.194128
186             sineCM PC 17^2  -463.012465
61    sineCM PC 2 sineCM PC 12  -438.806294
82               sineCM PC 4^2   413.403642
171  sineCM PC 12 sineCM PC 17  -410.781516
165  sineCM PC 11 sineCM PC 17  -408.920266
144   sineCM PC 9 sineCM PC 11   400.942858
69     sineCM PC 3 sineCM PC 5  -394.742249


Only Interaction Terms

In [67]:
df_features_scm = feature_sets[2].iloc[:, :-1]  # Select all columns except the last one

# Create PolynomialFeatures transformer
poly = PolynomialFeatures(degree=2,interaction_only=True,  include_bias=False)
#poly = PolynomialFeatures(degree=2, include_bias=False)

# Transform the features
df_poly = poly.fit_transform(df_features_scm)
df_poly = pd.DataFrame(df_poly, columns=poly.get_feature_names_out(df_features_scm.columns))
df_final = pd.concat([feature_sets[2], df_poly], axis=1)

model = BayesianRidge()
X = df_final.drop(columns=[target])
y = df_final[target]
    
model.fit(X, y)  # Fit model to extract alpha_ and lambda_
    
bayesridge_cv_mse = get_mean_cv_mse(model, df_final)
alpha_value = model.alpha_
lambda_value = model.lambda_  # Extract estimated lambda
print("MSE:", bayesridge_cv_mse)
print("Alpha:", alpha_value)
print("Lambda:", lambda_value)


MSE: 0.8197342569860249
Alpha: 1.2429563320758878
Lambda: 1.5514082794304762e-05


In [68]:
coefficients = model.coef_
feature_names = df_final.drop(columns=[target]).columns  
coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})
top_10_features = coef_df.reindex(coef_df.Coefficient.abs().sort_values(ascending=False).index).head(10)
print(top_10_features)


                       Feature  Coefficient
65     sineCM PC 3 sineCM PC 4   813.346920
84    sineCM PC 4 sineCM PC 10   599.090464
72    sineCM PC 3 sineCM PC 11   491.599009
82     sineCM PC 4 sineCM PC 8   483.408738
136   sineCM PC 9 sineCM PC 12   474.482121
135   sineCM PC 9 sineCM PC 11   425.918333
154  sineCM PC 11 sineCM PC 17  -401.497773
159  sineCM PC 12 sineCM PC 17  -374.786216
59    sineCM PC 2 sineCM PC 12  -370.099207
66     sineCM PC 3 sineCM PC 5  -364.777507


Stoich45 Int + SCM PCs

In [87]:
df_features_sscm = feature_sets[4].drop(columns=[target]) # Select all columns except the last one

# Create PolynomialFeatures transformer
poly = PolynomialFeatures(degree=2,interaction_only=True,  include_bias=False)
#poly = PolynomialFeatures(degree=2, include_bias=False)

# Transform the features
df_poly = poly.fit_transform(df_features_sscm)
df_poly = pd.DataFrame(df_poly, columns=poly.get_feature_names_out(df_features_sscm.columns))
df_final = pd.concat([feature_sets[4], df_poly], axis=1)

model = BayesianRidge()
X = df_final.drop(columns=[target])
y = df_final[target]
    
model.fit(X, y)  # Fit model to extract alpha_ and lambda_
    
bayesridge_cv_mse = get_mean_cv_mse(model, df_final)
alpha_value = model.alpha_
lambda_value = model.lambda_  # Extract estimated lambda
print("MSE:", bayesridge_cv_mse)
print("Alpha:", alpha_value)
print("Lambda:", lambda_value)


MSE: 0.6107909563651641
Alpha: 1.7341826120197854
Lambda: 40925108.651221685


In [84]:
df_features_sscm =feature_sets[4].drop(columns=[target])   # Select all columns except the last one

# Create PolynomialFeatures transformer
#poly = PolynomialFeatures(degree=2,interaction_only=True,  include_bias=False)
poly = PolynomialFeatures(degree=2, include_bias=False)

# Transform the features
df_poly = poly.fit_transform(df_features_sscm)
df_poly = pd.DataFrame(df_poly, columns=poly.get_feature_names_out(df_features_sscm.columns))


In [86]:
df_final



,atomic_num_standard_deviation,atomic_num_max,group_num_mean,period_num_mean,electronegativity_min,electron_affinity_mean,electron_affinity_geometric_mean,electron_affinity_standard_deviation,melting_mean,melting_geometric_mean,...,sineCM PC 14^2,sineCM PC 14 sineCM PC 15,sineCM PC 14 sineCM PC 16,sineCM PC 14 sineCM PC 17,sineCM PC 15^2,sineCM PC 15 sineCM PC 16,sineCM PC 15 sineCM PC 17,sineCM PC 16^2,sineCM PC 16 sineCM PC 17,sineCM PC 17^2
0,5.689851,29.0,7.960000,1.600000,1.90,100.252000,82.167123,45.559861,1143.143600,107.487344,...,1.089490e-05,3.786418e-06,5.924132e-06,-5.291655e-06,1.315933e-06,2.058875e-06,-1.839064e-06,3.221264e-06,-2.877352e-06,2.570158e-06
1,3.088689,19.0,13.700000,2.050000,0.82,110.030000,72.816144,57.821122,1199.022000,206.066393,...,2.264787e-05,-3.485902e-05,-3.538393e-05,5.461570e-06,5.365410e-05,5.446203e-05,-8.406308e-06,5.528212e-05,-8.532890e-06,1.317066e-06
2,6.272068,30.0,10.363636,1.818182,1.65,100.818182,0.000000,58.491030,1114.843636,141.329527,...,6.562173e-07,1.138244e-06,-1.060309e-06,1.509129e-07,1.974345e-06,-1.839163e-06,2.617666e-07,1.713236e-06,-2.438436e-07,3.470606e-08
3,8.169828,48.0,8.687500,1.656250,1.69,110.787500,0.000000,42.372240,1231.431562,122.519183,...,1.477867e-06,-9.167839e-06,1.824743e-06,-5.811074e-07,5.687202e-05,-1.131966e-05,3.604857e-06,2.253036e-06,-7.175014e-07,2.284954e-07
4,4.436880,27.0,9.097561,1.658537,1.88,104.460976,82.484786,49.421674,1552.056341,186.153480,...,2.143319e-07,-1.303685e-08,-3.510319e-07,-8.856839e-07,7.929733e-10,2.135170e-08,5.387219e-08,5.749186e-07,1.450569e-06,3.659912e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8349,6.968540,29.0,11.352941,2.000000,1.90,121.117647,67.620679,100.512201,1467.541765,253.964277,...,2.505958e-07,2.013156e-06,4.713419e-07,1.278293e-06,1.617264e-05,3.786515e-06,1.026913e-05,8.865401e-07,2.404321e-06,6.520587e-06
8350,11.575955,48.0,11.185185,2.000000,1.69,93.111111,0.000000,62.259852,935.485926,130.152404,...,1.522698e-07,6.090398e-08,7.005778e-07,1.610982e-08,2.436002e-08,2.802130e-07,6.443511e-09,3.223287e-06,7.411964e-08,1.704384e-09
8351,8.317703,48.0,10.903226,1.903226,1.69,102.548387,0.000000,64.328877,1786.626452,330.048989,...,1.013423e-04,9.536196e-05,5.231428e-05,2.134512e-05,8.973454e-05,4.922715e-05,2.008552e-05,2.700535e-05,1.101865e-05,4.495797e-06
8352,5.857798,30.0,9.739130,1.739130,1.65,116.756522,0.000000,44.348843,1706.767826,241.158462,...,8.464978e-05,2.371074e-05,-3.735797e-05,-1.670391e-05,6.641474e-06,-1.046412e-05,-4.678832e-06,1.648696e-05,7.371834e-06,3.296176e-06
